## Ejemplo 1: ClassicModels 
Trigger en PostgreSQL que mantenga actualizada la columna `quantityInStock` de la tabla `products` cuando se realizan operaciones `INSERT`, `UPDATE` o `DELETE` en la tabla `orderDetails`. Además, se asegura de impedir la actualización si `quantityInStock` resulta menor a 0.

### Código del Trigger y la Función

```sql
-- Crear función para manejar el trigger
CREATE OR REPLACE FUNCTION update_quantity_in_stock()
RETURNS TRIGGER AS $$
BEGIN
    -- Manejar caso de INSERT
    IF TG_OP = 'INSERT' THEN
        -- Verificar que haya suficiente inventario antes de insertar
        IF (SELECT quantityInStock FROM products WHERE productCode = NEW.productCode) < NEW.quantityOrdered THEN
            RAISE EXCEPTION 'No se puede procesar el pedido. Inventario insuficiente.';
        END IF;
        -- Reducir inventario
        UPDATE products
        SET quantityInStock = quantityInStock - NEW.quantityOrdered
        WHERE productCode = NEW.productCode;
    END IF;

    -- Manejar caso de DELETE
    IF TG_OP = 'DELETE' THEN
        -- Restaurar inventario
        UPDATE products
        SET quantityInStock = quantityInStock + OLD.quantityOrdered
        WHERE productCode = OLD.productCode;
    END IF;

    -- Manejar caso de UPDATE
    IF TG_OP = 'UPDATE' THEN
        -- Restaurar inventario con el pedido anterior
        UPDATE products
        SET quantityInStock = quantityInStock + OLD.quantityOrdered
        WHERE productCode = OLD.productCode;

        -- Verificar que haya suficiente inventario antes de actualizar
        IF (SELECT quantityInStock FROM products WHERE productCode = NEW.productCode) < NEW.quantityOrdered THEN
            RAISE EXCEPTION 'No se puede actualizar el pedido. Inventario insuficiente.';
        END IF;

        -- Reducir inventario con el nuevo pedido
        UPDATE products
        SET quantityInStock = quantityInStock - NEW.quantityOrdered
        WHERE productCode = NEW.productCode;
    END IF;

    RETURN NULL; -- Los triggers AFTER no requieren devolver nada
END;
$$ LANGUAGE plpgsql;

-- Crear el trigger en la tabla orderDetails
CREATE TRIGGER trg_update_quantity_in_stock
AFTER INSERT OR UPDATE OR DELETE ON orderDetails
FOR EACH ROW
EXECUTE FUNCTION update_quantity_in_stock();
```

### Explicación del Código
1. **Función `update_quantity_in_stock`:**
   - Es una función que maneja los eventos `INSERT`, `UPDATE` y `DELETE`.
   - Se verifica el estado del inventario antes de reducirlo en los casos de `INSERT` y `UPDATE`.
   - El inventario se restaura cuando se elimina un pedido (`DELETE`) o se actualiza un pedido existente (`UPDATE`).
   - Se arroja un error (`RAISE EXCEPTION`) si la operación haría que el inventario sea menor que 0.

2. **Trigger `trg_update_quantity_in_stock`:**
   - Se activa después de cada operación `INSERT`, `UPDATE` o `DELETE` en la tabla `orderDetails`.
   - Invoca la función `update_quantity_in_stock`.

### Ejemplo de Uso
1. **Insertar un pedido:**
   ```sql
   INSERT INTO orderDetails (orderNumber, productCode, quantityOrdered, priceEach, orderLineNumber)
   VALUES (10103, 'S10_1678', 20, 48.81, 1);
   ```

2. **Actualizar un pedido:**
   ```sql
   UPDATE orderDetails
   SET quantityOrdered = 50
   WHERE orderNumber = 10103 AND productCode = 'S10_1678';
   ```

3. **Eliminar un pedido:**
   ```sql
   DELETE FROM orderDetails
   WHERE orderNumber = 10103 AND productCode = 'S10_1678';
   ```

Este trigger garantiza la integridad de los datos y evita situaciones de inventario negativo. 